In [1]:
import PySimpleGUI as sg
import modulo1_projeto as fil
from PIL import Image

In [2]:
BD=fil.carregarBD('cinemaATP (1).json') #path to file

In [3]:
def janelaErro(mensagem):
    sg.theme('DarkBlack')
    interface = [
        [sg.Text(mensagem)],
        [sg.Button('Ok')]
    ]
    
    window = sg.Window(title='ERRO').Layout(interface)
    
    stop=False
    while not stop:
        event, values = window.read()
        if event == 'Ok' or event == sg.WIN_CLOSED:
            stop=True
    window.close()

In [4]:
def janelaGenero(bd):
    sg.theme('DarkBlack')
    filmes=[]
    genres=[]
    g=fil.distribGenero(bd)
    for chave in g.keys():
        genres.append(chave)
        genres.sort()
    generos=[
        [sg.Button('Procurar'),sg.Button('Listar todos os filmes')],
        [sg.Listbox(values=genres,key='-gnr',size=(25,30))],
        [sg.Button('Distribuição por Géneros',size=(23))]
    ]
    interface=[
        [sg.Column(generos),
        sg.Listbox(values=filmes,key='-genero',size=(200,150)),
        sg.Canvas(key='-canvas')]
    ]
    
    
    window=sg.Window('Géneros',size=(900,600)).Layout(interface)

    stop=False
    while not stop:
        event,values=window.read()
        if event == sg.WIN_CLOSED:
            stop = True
        elif event == 'Procurar':
            if len(values['-gnr'])>0:
                filmes=fil.genero(bd,values['-gnr'][0])
                window.find_element('-genero').update(filmes)
        elif event == 'Listar todos os filmes':
            filmes=fil.indGeneros(BD)
            window.find_element('-genero').update(filmes)
        elif event == 'Distribuição por Géneros':
            fig=fil.plotDistribGenero(bd)
            fig_canvas_agg=fil.draw_figure(window['-canvas'].TKCanvas,fig)
    window.close()

In [35]:
def janelaAtor(bd):
    sg.theme('DarkBlack')
    filmes=[]
    interface=[
        [sg.Input('Escreva o nome do ator',size=(83),key='-ator'),sg.Button('Pesquisar'),sg.Button('Top 10'),sg.Button('Listar todos os atores')],
        [sg.Listbox(values=filmes,key='-cast',size=(300,50))],
        [sg.Canvas(key='-canvas')]
    ]
    
    window=sg.Window('Atores',size=(900,600)).Layout(interface)

    stop=False
    while not stop:
        event,values=window.read()
        if event == sg.WIN_CLOSED:
            stop = True
        elif event == 'Pesquisar':
            if values['-ator'] != 'Escreva o nome do ator':
                filmes=fil.ator(bd,values['-ator'])
                if filmes==[]:
                    janelaErro('Ator não encontrado')
                else:
                    window.find_element('-cast').update(filmes)
            else:
                janelaErro('Insira o nome do ator')
        elif event == 'Listar todos os atores':
            filmes=fil.indAtores(BD)
            window.find_element('-cast').update(filmes)
        elif event == 'Top 10':
            fig=fil.plotDistribAtor(bd)
            fig_canvas_agg=fil.draw_figure(window['-canvas'].TKCanvas,fig)
    window.close()

In [36]:
def janelaInserir(bd):
    sg.theme('DarkBlack')
    interface=[
        [sg.Text('f'+str(len(bd)+1))],
        [sg.Text('Título'),sg.Input(size=(48),key='-title')],
        [sg.Text('Ano'),sg.Input(size=(49),key='-year')],
        [sg.Text('Atores'),sg.Input(size=(47),key='-cast')],
        [sg.Text('Géneros'),sg.Input(key='-genres')],
        [sg.Button('Inserir Filme')]
        ]
    window=sg.Window('Inserir').Layout(interface)
    stop=False
    while not stop:
        event, values = window.read() 
        if event == sg.WIN_CLOSED: 
            stop = True
        elif event == 'Inserir Filme':
            title=values['-title']
            year=values['-year']
            cast=values['-cast'].split(', ')
            genres=values['-genres'].split(', ')
            fil.inserirFilme(bd,title,year,cast,genres)
            fil.guardarBD(BD,'cinemaATP (1).json')
            window.close()
    window.close()
    

In [37]:
def janelaAlterar(bd,filme):
    sg.theme('DarkBlack')
    interface=[
        [sg.Text('Id'),sg.Input(filme['id'],size=(51),key='-id')],
        [sg.Text('Título'),sg.Input(filme['title'],size=(48),key='-title')],
        [sg.Text('Ano'),sg.Input(filme['year'],size=(49),key='-year')],
        [sg.Text('Atores'),sg.Input(filme['cast'],size=(47),key='-cast')],
        [sg.Text('Géneros'),sg.Input(filme['genres'],key='-genres')],
        [sg.Button('Alterar Filme'),sg.Button('Remover Filme')]
        ]
    window=sg.Window('Alterar').Layout(interface)
    stop=False
    while not stop:
        event, values = window.read() 
        if event == sg.WIN_CLOSED: 
            stop = True
        elif event == 'Alterar Filme':
            bd.remove(filme)
            title=values['-title']
            year=values['-year']
            cast=values['-cast'].split(', ')
            genres=values['-genres'].split(', ')
            id=values['-id']
            fil.alterarFilme(BD,id,title,year,cast,genres)
            fil.guardarBD(BD,'cinemaATP (1).json')
            window.close()
        elif event=='Remover Filme':
            bd.remove(filme)
            fil.guardarBD(BD,'cinemaATP (1).json')
            window.close()   
    window.close()

In [38]:
sg.theme('DarkBlack')

filmes=[]

menu=[
    [sg.Text('Número de filmes: '),sg.Text(key='-len-')],
    [sg.Button('Carregar Filmes',size=(30,2))],
    [sg.Button('Consultar Género',size=(30,2))],
    [sg.Button('Consultar Ator',size=(30,2))],
    [sg.Button('Inserir Filme',size=(30,2))],
    [sg.Button('Alterar Filme',size=(30,2))],
    [sg.Button('Sair',size=(30,2))]
    
]

data_viewer=[
    [sg.Input('Pesquise um filme',key='pf',size=(52)),sg.Button('Pesquisar por ID'),sg.Button('Pesquisar por Título'),sg.Button('Informações')],
    [sg.Listbox(values=filmes,size=(98,40),key='-list-')]
]


interface=[
    [sg.Column(menu),
    sg.VSeparator(),
    sg.Column(data_viewer)]
]

window=sg.Window('UMovies',size=(1027,700)).Layout(interface)

stop=False
while not stop:
    event,values=window.read()
    if event == 'Sair' or event == sg.WIN_CLOSED:
        stop = True
    elif event == 'Carregar Filmes':
        filmes=fil.listarFilmes(BD)
        window['-len-'].update(str(len(BD)))
        window.find_element('-list-').update(visible=True)
        window.find_element('-list-').update(filmes)
    elif event == 'Consultar Género':
        janelaGenero(BD)
    elif event == 'Consultar Ator':
        janelaAtor(BD)
    elif event == 'Pesquisar por ID':
        if values['pf']!='Pesquise um filme':
            filmes=fil.consultarID(BD,values['pf'])
            if filmes == []:
                janelaErro('ERRO: ID inválido!')
            window.find_element('-list-').update(values=filmes)
        else:
            janelaErro('ERRO: Insira um ID!')
    elif event == 'Pesquisar por Título':
        if values['pf']!='Pesquise um filme':
            filmes=fil.consultarNome(BD,values['pf'])
            if filmes == []:
                janelaErro('ERRO: Título inválido!')
            window.find_element('-list-').update(values=filmes)
        else:
            janelaErro('ERRO: Insira um título!')
    elif event == 'Inserir Filme':
        janelaInserir(BD)
        filmes=fil.listarFilmes(BD)
        window['-len-'].update(str(len(BD)))
        window.find_element('-list-').update(filmes)
    elif event == 'Alterar Filme':
        if len(values['-list-'])>0:
            if type(values['-list-'][0])==dict:
                filme=values['-list-'][0]
                janelaAlterar(BD,filme)
            else:
                for filme in BD:
                    for elem in values['-list-'][0]:
                        if elem==filme['id']:
                            movie=filme
                            janelaAlterar(BD,movie)
        else:
            janelaErro('ERRO: Primeiro selecione um filme')
        filmes=fil.listarFilmes(BD)
        window['-len-'].update(str(len(BD)))
        window.find_element('-list-').update(filmes)
    elif event == 'Informações':
        if len(values['-list-'])>0:
            if type(values['-list-'][0])==dict:
                informacoes=list(values['-list-'][0].items())
                filmes=informacoes
                window.find_element('-list-').update(filmes)
            else:
                for filme in BD:
                    for elem in values['-list-'][0]:
                        if elem==filme['id']:
                            movie=filme
                            informacoes=list(movie.items())
                            filmes=informacoes
                            window.find_element('-list-').update(filmes)
        else:
            janelaErro('ERRO: Primeiro selecione um filme')    
    fil.guardarBD(BD,'cinemaATP (1).json') #path to file
        
    
window.close()